In [9]:
import numpy as np
import pandas as pd
import spacy

In [14]:
the_seed = 42
torch.manual_seed(the_seed)
torch.cuda.manual_seed(the_seed)
np.random.seed(the_seed)

Read data. To be generic we do not hard code variable names as their language name. Let's simply call them source and destination

In [19]:
# set language here
language_meta = {
    "en": {
        "spacy": "en_core_web_sm",
        "input_file": "news-commentary-v13.zh-en.en"
    },
    "zh": {
        "spacy": "zh_core_web_sm",
        "input_file": "news-commentary-v13.zh-en.zh"
    }
}


source_language = "en"
dest_language = "zh"

source_nlp = spacy.load(language_meta[source_language]["spacy"])
dest_nlp = spacy.load(language_meta[dest_language]["spacy"])
source_file = language_meta[source_language]["input_file"]
dest_file = language_meta[dest_language]["input_file"]

tokens = dest_nlp.tokenizer("秦始皇是一个好国王？")

#[token.text for token in en_nlp.tokenizer(string)]

print([t.text for t in tokens])



['秦始皇', '是', '一个', '好', '国王', '？']


In [7]:
src_lines = open(source_file).readlines()
dest_lines = open(dest_file).readlines()

# Shuffle the DataFrame rows
df = pd.DataFrame(list(zip(src_lines, dest_lines)), columns=['source', 'destination'])
df = df.sample(frac=1).reset_index(drop=True)

train_offset = int(len(df) * 0.8)
test_offset = int(len(df) * 0.9)

train_set = df.iloc[:train_offset]
test_set = df.iloc[train_offset: test_offset]
valid_set = df.iloc[test_offset:]

print(valid_set.iloc[0])




source         With reforms of the traditionally Western-domi...
destination    随着传统西方主导的改革陷入僵局，人们转而创造其他替代性措施； 例如中国带头创建的亚洲基础设施...
Name: 227499, dtype: object
